<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# This notebook creates an image with additional Applications that can be used alongside Jupyter

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)


### Configuration

In [ ]:
image_name = 'jupyter-with-apps'
folder_name = '/efs/shared/samples/notebooks/M-Admin/extra_user_apps/'

In [ ]:
%cd $folder_name

### Build Image

Lets see how  orbit build image works...

get our orbit env and team names

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [ ]:
repository_name = (f"orbit-{env_name}/users/{image_name}")
repository_name

In [ ]:
!aws ecr delete-repository --repository-name $repository_name --force

adding a file to our docker as an example

In [ ]:
pwd = %pwd
pwd

Now lets run the command

In [ ]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name 
output

Lets get the image address from the output of the previous command

In [ ]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

In [ ]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

### Building the profile for the Image

In [ ]:
import json

import json
profile = [
    {
        "display_name": "CodeServer IDE",
        "slug": "code-server",
        "description": "A VSCode-based cloud IDE in your K8 team",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 3,
            "cpu_limit": 3,
            "mem_guarantee": "4G",
            "mem_limit": "4G",
            "default_url": "/code-server"
        },
    },
    {
        "display_name": "WorkStation",
        "slug": "linuxdesktop",
        "description": "Linux Desktop with apps in your K8 team",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 3,
            "cpu_limit": 3,
            "mem_guarantee": "4G",
            "mem_limit": "4G",
            "default_url": "/desktop"
        },
    },
    {
        "display_name": "Jupyter with Apps",
        "slug": "jupyterapps",
        "description": "Jupyter Notebook with additional apps",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 3,
            "cpu_limit": 3,
            "mem_guarantee": "4G",
            "mem_limit": "4G"
        },
    }
]

with open("profile.json", 'w') as f:
    json.dump(profile, f)


with open("profile.json", 'w') as f:
    json.dump(profile, f)


In [ ]:
!cat profile.json

In [ ]:
!orbit build profile --debug --env $env_name --team lake-user profile.json

## Users can now use the new profile to use new apps

***

## Adding Extensions
The extension SQLTools requires additional drivers to be installed.  For example to connect to Redshift, there is a driver available (mtxr.sqltools-driver-pg) from the VS Marketplace that works with SQLTools.  But, it has to be installed separately as it is not included in the SQLTools installation.  You can download the driver by fetching the VSIX binary and installing manually.  A script is provided at /home/jovyan/.local/share/code-server/scripts that can fetch the VSIX and install it.  

Or, you can do the following:

1. Fetch the VSIX by any means available (options)
    1. curl https://marketplace.visualstudio.com/_apis/public/gallery/publishers/mtxr/vsextensions/sqltools-driver-pg/0.2.0/vspackage
    2. Browse to the marketplace and download sql-tools.driver-pg driver
2. Install the driver (options)
    1. from command palatte, look for 'Install From VSIX' , browse to the downloaded VSIX file and install
    2. run commanndline code-server --install-extension location_of_file.vsix --force
3. Restart VSCode
